In [ ]:
import smtplib
import re
import dns.resolver
from time import time, sleep
import socket
import pandas as pd  # For handling Excel input/output

# Validate email syntax using regex
def is_valid_syntax(email):
    email_regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return re.match(email_regex, email) is not None

# Verify the domain has MX records
def has_valid_mx_record(domain):
    try:
        dns.resolver.resolve(domain, 'MX')
        return True
    except (dns.resolver.NoAnswer, dns.resolver.NXDOMAIN):
        return False

# Perform SMTP validation using Gmail's SMTP server
def is_email_valid_smtp(email, gmail_user, gmail_app_password):
    try:
        # Step 1: Check email syntax
        if not is_valid_syntax(email):
            return False
        
        # Step 2: Validate the domain's MX records
        domain = email.split('@')[1]
        if not has_valid_mx_record(domain):
            return False

        # Step 3: SMTP-level validation
        server = smtplib.SMTP("smtp.gmail.com", 587, timeout=30)
        server.starttls()
        server.login(gmail_user, gmail_app_password)  # Authenticate
        server.helo()
        server.mail(gmail_user)  # Set the sender email
        code, message = server.rcpt(email)  # Verify the recipient email

        server.quit()
        return code == 250  # Check SMTP response code
    except Exception as e:
        print(f"Error validating email {email}: {e}")
        return False

# Main function to validate emails from an Excel file
def validate_emails(input_file, email_column, gmail_user, gmail_app_password, start_row=None, end_row=None):
    start_time = time()
    df = pd.read_excel(input_file)  # Load data from Excel file

    # Slice the dataset for the specified range
    if start_row is not None and end_row is not None:
        df = df.iloc[start_row:end_row]

    # Prepare output file names
    base_name = input_file.split('.')[0]
    output_file_valid = f"{base_name} valid emails {start_row + 1}-{end_row}.xlsx"
    output_file_invalid = f"{base_name} invalid emails {start_row + 1}-{end_row}.xlsx"

    validated_data = []
    invalid_data = []

    # Iterate through emails and validate
    for index, row in df.iterrows():
        email = row[email_column]
        email_start_time = time()
        is_valid = is_email_valid_smtp(email, gmail_user, gmail_app_password)
        time_taken = time() - email_start_time

        if is_valid:
            print(f"{email}: Valid, Time taken: {time_taken:.2f} seconds")
            validated_data.append({**row.to_dict(), "Validation Status": "Valid", "Validation Time (s)": time_taken})
        else:
            print(f"{email}: Invalid, Time taken: {time_taken:.2f} seconds")
            invalid_data.append({**row.to_dict(), "Validation Status": "Invalid", "Validation Time (s)": time_taken})

        sleep(30)  # Pause to avoid server throttling

    # Save valid emails to Excel
    if validated_data:
        pd.DataFrame(validated_data).to_excel(output_file_valid, index=False)
        print(f"Validated emails saved to {output_file_valid}")
    
    # Save invalid emails to Excel
    if invalid_data:
        pd.DataFrame(invalid_data).to_excel(output_file_invalid, index=False)
        print(f"Invalid emails saved to {output_file_invalid}")
    
    total_time = (time() - start_time) / 3600
    print(f"Total time taken: {total_time:.2f} hours")

if __name__ == "__main__":
    gmail_user = "xxxxxxxxx@xxxx.com"
    gmail_app_password = "xxxxxxxxxx"
    input_file = "xxxxxxxxx.xlsx"
    email_column = "Email"

    start_row = 0
    end_row = 1000  # Adjust as needed

    validate_emails(input_file, email_column, gmail_user, gmail_app_password, start_row, end_row)
